# Data Structures


In [ ]:
import uxarray as ux
import xarray as xr

UXarray provides three data structures for interacting with unstructured grids and the data variables that reside on them:

1. **ux.Grid**: Stores the grid representation (i.e. coordinates, connectivity information, etc.)
2. **ux.UxDataset**: One or more data variable that resided on a grid.
3. **ux.UxDataArray**: A single data variable that resides on a grid 


## Grid and Data Files

When working with unstructured grid datasets, the grid definition is typically stored separately from any data variables. 

For example, the dataset we're using in this example is made up of two files: 1 grid definition and 1 data file.

```
quad-hexagon
│   grid.nc
│   data.nc
```

In [ ]:
grid_path = "../../test/meshfiles/ugrid/quad-hexagon/grid.nc"
data_path = "../../test/meshfiles/ugrid/quad-hexagon/data.nc"

Additionally, there may be multiple data files that are mapped to the same unstructured grid (think climate model history files). Using our sample dataset, this may look something like this:

```
quad-hexagon
│   grid.nc
│   data1.nc
|   data2.nc
|   data3.nc
```

We can store these paths as a list (in this case we simply repeat the original data file to imitate having 4 separate data files)

In [ ]:
multiple_data_paths = [data_path for i in range(3)]

## Grid

UXarray's `Grid` class is used for representing unstructured grids in terms of their coordinates and connectivity information.

### Creating a Grid

The recommended way to construct a `Grid` is by using the `ux.open_grid()` method, which takes in a grid file path, detects the input grid format, and parses and encodes the provided coordinates and connectivity into the UGRID conventions. Details on supported grid formats and what variables are parsed can be found in other parts of this user guide.

In [ ]:
uxgrid = ux.open_grid(grid_path)
uxgrid

### Grid Attributes

Each `Grid` contains multiple dimensions, coordinates, connectivity, and descriptor variables that represent an unstructured grid. All currently avaliable variables can be viewed by printing a `Grid`


In [ ]:
uxgrid

These variables can be accessed as an attribute of the `Grid` class. 

In [ ]:
uxgrid.n_node

In [ ]:
uxgrid.face_node_connectivity

## UxDataset

Up to this point, we've exclusively looked at the unstructured grid without any data variables mapped to it. When working with one or more data variables, they are linked to a grid through a `UxDataset` instance.

When pairing data variables with an unstructured grid files, they can be opened together using the `ux.open_dataset()` method, which returns a ``UxDataset``


### Opening a Single Data File

We can load a pair of grid and data files using the `ux.open_dataset()` method.



In [ ]:
uxds = ux.open_dataset(grid_path, data_path)
uxds

### Opening Multiple Data Files

When working with multiple data paths, we can open them using the `ux.open_mfdataset()` method. 

In [ ]:
uxds_multi = ux.open_mfdataset(grid_path, multiple_data_paths, combine ='nested', concat_dim='time')
uxds_multi

## Grid Accessor

Each `UxDataset` (and in the next section `UxDataArray`) is linked to a `Grid` instance, which contain the unstructured grid information

In [ ]:
uxds.uxgrid

## UxDataArray

While a `UxDataset` represents one or more data variables linked to some unstructured grid, a `UxDataArray` represent a single data variable. Alternatively, one can think of a `UxDataset` as a collection of one or more `UxDataArray` instances.

In our sample dataset, we have a variable called `t2m`, which can be used to index our `UxDataset`


In [ ]:
uxds['t2m']

As mentioned before, each `UxDataArray` is linked to a `Grid` instance.

In [ ]:
uxds['t2m'].uxgrid

This Grid is identical to the one linked to the `UxDataset`. There is a single `Grid` that is shared by all data variables.

Just like with Xarray, we can perform various operations on our data variable

In [ ]:
uxds['t2m'].uxgrid == uxds.uxgrid

In [ ]:
uxds['t2m'].min()

In [ ]:
uxds['t2m'].mean()

UXarray also provides custom data analysis operators which are explored in further sections of this user guide

In [ ]:
uxds['t2m'].gradient()

## Inheritance from Xarray

For those that are familiar with Xarray, the naming of the methods and data structures looks familiar. UXarray aims to provide a familiar experience to Xarray by inheriting the `xr.Dataset` and `xr.DataArray` objects and linking them to an instance of a `Grid` class to provide grid-aware implementations.

We can observe this inheritance by checking for subclassing.

In [ ]:
issubclass(ux.UxDataset, xr.Dataset)

In [ ]:
issubclass(ux.UxDataArray, xr.DataArray)

## Overloaded Methods

With subclassing, all methods are inherited by default. This means that while most will execute, their output may not be as expected. We have re-implemented and added many new methods to provide grid-aware implementations of their Xarray counterparts. 

These are discussed in detail in the next sections, but one notable example are all the plotting methods are all custom to support unstructured grid visualuzation.


In [ ]:
uxds['t2m'].plot(fig_size=150, colorbar=False, backend='matplotlib')